# Part 3: Word Representations and Lexical Similarities

This part has 18 points in total.

Here we will compare different measures of semantic similarity between words: (1) WordNet depth distance (2) cosine similarity of words using a given GloVe model and (3) Resnet50 image features.

For more reading on vector semantics got to Chapter 6, sections 6.4 and 6.8:
https://web.stanford.edu/~jurafsky/slp3/6.pdf

To learn about Wordnet: https://www.nltk.org/howto/wordnet.html

For additional Wordnet discussions see Chapter 19: https://web.stanford.edu/~jurafsky/slp3/19.pdf

The GloVe word embeddings are described in [this paper](https://nlp.stanford.edu/projects/glove/)

Resnet50: Deep Residual Learning for Image Recognition are described in [this paper](https://arxiv.org/abs/1512.03385)

## Part 3.1: Semantic similarity with WordNet

In [262]:
# load wordnet
from nltk.corpus import wordnet as wn

# load word-vector glov
import gensim.downloader as gensim_api
glove_model = gensim_api.load("glove-wiki-gigaword-50")

from itertools import combinations, product
from scipy.stats import spearmanr
import numpy as np

In [263]:
some_words = ['car', 'dog', 'banana', 'delicious', 'baguette', 'jumping', 'hugging', 'election']

### Explore Word Representations in English WordNet (+3pt)

In [264]:
# For each word above print their synsets
# for each synset print all lemmas, hypernyms, hyponyms

for word in some_words:
    synsets = wn.synsets(word)
    print(f'---Word: {word} ---')
    
    for synset in synsets:
        synset_name = synset.name()
        print(f'Synset: {synset_name}')
        
        synset_info = wn.synset(synset_name)
        lemmas = ', '.join(map(lambda lemma: lemma.name(), synset_info.lemmas()))
        print(f'LEmmas: {lemmas}')
        
        hypernyms = ', '.join(map(lambda hypernym: hypernym.name(), synset_info.hypernyms()))
        print(f'Hypernyms: {hypernyms}')
        
        
        hyponyms = ', '.join(map(lambda hyponym: hyponym.name(), synset_info.hyponyms()))
        print(f'Hyponyms: {hyponyms}\n')
        print(f'____________________________')
        

---Word: car ---
Synset: car.n.01
LEmmas: car, auto, automobile, machine, motorcar
Hypernyms: motor_vehicle.n.01
Hyponyms: ambulance.n.01, beach_wagon.n.01, bus.n.04, cab.n.03, compact.n.03, convertible.n.01, coupe.n.01, cruiser.n.01, electric.n.01, gas_guzzler.n.01, hardtop.n.01, hatchback.n.01, horseless_carriage.n.01, hot_rod.n.01, jeep.n.01, limousine.n.01, loaner.n.02, minicar.n.01, minivan.n.01, model_t.n.01, pace_car.n.01, racer.n.02, roadster.n.01, sedan.n.01, sport_utility.n.01, sports_car.n.01, stanley_steamer.n.01, stock_car.n.01, subcompact.n.01, touring_car.n.01, used-car.n.01

____________________________
Synset: car.n.02
LEmmas: car, railcar, railway_car, railroad_car
Hypernyms: wheeled_vehicle.n.01
Hyponyms: baggage_car.n.01, cabin_car.n.01, club_car.n.01, freight_car.n.01, guard's_van.n.01, handcar.n.01, mail_car.n.01, passenger_car.n.01, slip_coach.n.01, tender.n.04, van.n.03

____________________________
Synset: car.n.03
LEmmas: car, gondola
Hypernyms: compartment.n.

#### Measure The Lexical Similarity (+3pt)

In [265]:
# Wu-Palmer Similarity is a measure of similarity between to sense based on their depth distance. 
#
# For each pair of words, find their closes sense based on Wu-Palmer Similarity.
# List all word pairs and their highest possible wup_similarity. 
# Use wn.wup_similarity(s1, s2) and itertools (combinations and product).
# if there is no connection between two words, put 0.
w_pairs = []
wn_sims = []
for word1, word2 in combinations(some_words, 2):
    w_pairs.append((word1, word2))
    w1_synsets = wn.synsets(word1)
    w2_synsets = wn.synsets(word2)
    
    max_sim = 0
    for synset1, synset2 in product(w1_synsets, w2_synsets):
        sim = wn.wup_similarity(synset1, synset2)
        
        if sim > max_sim:
            max_sim = sim
    wn_sims.append(max_sim)
    print(f"{word1:9} {word2:9} {max_sim:6.3f}")
    ######################
max_sim = max(wn_sims)
max_sim_index = wn_sims.index(max_sim)
max_sim_pair = w_pairs[max_sim_index]

print(f'\nMost Similiar words')
##Pairing
print(f'Pair: {max_sim_pair}, Score: {max_sim}')

# which word pair are the most similar words?
# Answear: banana    delicious  0.750

car       dog        0.667
car       banana     0.421
car       delicious  0.364
car       baguette   0.211
car       jumping    0.167
car       hugging    0.235
car       election   0.133
dog       banana     0.632
dog       delicious  0.556
dog       baguette   0.556
dog       jumping    0.333
dog       hugging    0.286
dog       election   0.182
banana    delicious  0.750
banana    baguette   0.556
banana    jumping    0.167
banana    hugging    0.250
banana    election   0.143
delicious baguette   0.500
delicious jumping    0.500
delicious hugging    0.400
delicious election   0.222
baguette  jumping    0.154
baguette  hugging    0.222
baguette  election   0.125
jumping   hugging    0.400
jumping   election   0.667
hugging   election   0.200

Most Similiar words
Pair: ('banana', 'delicious'), Score: 0.75


## Part 3.2: Semantic similarity with GloVe and comparison with WordNet

### Measure the similarities on GloVe Word Vectors

In [266]:
glov_sims = []
for word1, word2 in combinations(some_words, 2):
    max_sim = glove_model.similarity(word1, word2)
    glov_sims.append(max_sim)
    print(f"{word1:9} {word2:9} {max_sim:6.3f}")


car       dog        0.464
car       banana     0.219
car       delicious  0.068
car       baguette   0.046
car       jumping    0.516
car       hugging    0.278
car       election   0.333
dog       banana     0.333
dog       delicious  0.404
dog       baguette   0.018
dog       jumping    0.539
dog       hugging    0.410
dog       election   0.181
banana    delicious  0.487
banana    baguette   0.450
banana    jumping    0.108
banana    hugging    0.127
banana    election   0.164
delicious baguette   0.421
delicious jumping    0.042
delicious hugging    0.142
delicious election   0.028
baguette  jumping   -0.075
baguette  hugging    0.161
baguette  election  -0.091
jumping   hugging    0.447
jumping   election   0.206
hugging   election  -0.076


#### Examine if two measures correlate

In [267]:
# a correlation coefficent of two lists
print("Spearman's rho", spearmanr(glov_sims, wn_sims))

# Higher correlation (closer to 1.0) means two measures agree with each other.

Spearman's rho SpearmanrResult(correlation=0.4222499442309076, pvalue=0.02519986065189366)


How do the two similarities compare? (+3pt)

In [268]:
# Write your answer here
#There are diffrences with the two similarity models Glove and Wu. Glove_model uses cosine similarity, calculated
#distance of vectors in the feature "plane"/space. The logic goes that if distance is 0 the vectors between the features
#are 90 degrees, therefore no similarity. 

#Wu-Palmer uses depth of the hypernym tree and calculates the similarity between the words. Its almost like a BTS-tree
#where it has a root. Therefore there is a common root for all words and it cant be negative. 

#The next segment explains it more etensively, dependencies/similarities are explained as Vectors using cosine_sim.

### Word Vector Representations in GloVe

In [269]:
# Each word is represented as a vector:
print('dog =', glove_model['dog'])

# matrix of all word vectors is trained as parameters of a language model:
# P( target_word | context_word ) = f(word, context ; params)
#
# Words in a same sentence and in close proximity are in context of each other.

dog = [ 0.11008   -0.38781   -0.57615   -0.27714    0.70521    0.53994
 -1.0786    -0.40146    1.1504    -0.5678     0.0038977  0.52878
  0.64561    0.47262    0.48549   -0.18407    0.1801     0.91397
 -1.1979    -0.5778    -0.37985    0.33606    0.772      0.75555
  0.45506   -1.7671    -1.0503     0.42566    0.41893   -0.68327
  1.5673     0.27685   -0.61708    0.64638   -0.076996   0.37118
  0.1308    -0.45137    0.25398   -0.74392   -0.086199   0.24068
 -0.64819    0.83549    1.2502    -0.51379    0.04224   -0.88118
  0.7158     0.38519  ]


### Implement Cosine Similarity (+3pt)

In [270]:
# based on equation 6.10 J&M (2019)
# https://web.stanford.edu/~jurafsky/slp3/6.pdf
#
def cosine_sim(v1, v2):
    def dot(v1, v2):
        return sum(v1*v2)
    
    def length(v):
        return dot(v, v) ** 0.5
    
    return dot(v1, v2)/ (length(v1) * length(v2))

cosine_sim(glove_model['car'], glove_model['automobile'])

0.6956217371771922

### Implement top-n most similar words (+3pt)

In [271]:
# search in glove_model:
def top_n(word, n):
    # example: top_n('dog', 3) =  
    #[('cat', 0.9218005537986755),
    # ('dogs', 0.8513159155845642),
    # ('horse', 0.7907583713531494)]
    # similar to glove_model.most_similar('dog', topn=3)
    sims = []
    for sim_w in glove_model.index_to_key:
        if sim_w == word:
            continue
            
        vector_w = glove_model[word]
        sim_vec_w = glove_model[sim_w]
        
        sim = cosine_sim(vector_w, sim_vec_w)
        pair = sim_w, sim
        sims.append(pair)
    
    sims.sort(key = lambda pair: pair[1], reverse = True)
    out = sims[:n]
    return out

#To test most sim. w to dog
for word in some_words:
    print(f'Most similar words to {word}:', top_n(word, 3))


Most similar words to car: [('truck', 0.9208586111508545), ('cars', 0.8870189568390014), ('vehicle', 0.8833684160157461)]
Most similar words to dog: [('cat', 0.9218005180563862), ('dogs', 0.851315860708305), ('horse', 0.7907583073303338)]
Most similar words to banana: [('bananas', 0.8152027815146712), ('coconut', 0.7872509691547495), ('pineapple', 0.7579815027266895)]
Most similar words to delicious: [('tasty', 0.9297150293689781), ('savory', 0.8695854940874334), ('spicy', 0.8472648526796178)]
Most similar words to baguette: [('brioche', 0.7667232075941974), ('baguettes', 0.7605002107488397), ('focaccia', 0.7604009696564551)]
Most similar words to jumping: [('jump', 0.8335474991521888), ('jumps', 0.7868058809019087), ('climbing', 0.7549090228759915)]
Most similar words to hugging: [('kissed', 0.7903850508373103), ('kissing', 0.7728414628740475), ('hugged', 0.765109992226354)]
Most similar words to election: [('elections', 0.9582603048189698), ('polls', 0.890477251299923), ('vote', 0.87

## Part 3.3: Semantic similarity with visual features (ResNet)


### Measure the similarities with the ResNet vectors

In this part we will use visual features of images representing these objects. If you are interested how we extract these features have a look at `visual-feature-extraction.ipynb` but understanding that notebook is not necessary to complete this part as we have saved them for you they are loaded in the code below.

In [272]:
# run the feature extractor on all images
# make sure that the order of features is identical to the order of words (variable some_words)

In [273]:
some_words

['car',
 'dog',
 'banana',
 'delicious',
 'baguette',
 'jumping',
 'hugging',
 'election']

In [274]:
object_indices = {v:k for k,v in enumerate(some_words)}
print(object_indices)

{'car': 0, 'dog': 1, 'banana': 2, 'delicious': 3, 'baguette': 4, 'jumping': 5, 'hugging': 6, 'election': 7}


In [275]:
import torch
image_features = torch.load('image_features.pt')

In [276]:
image_features.shape

torch.Size([8, 2048])

In [277]:
from sklearn.metrics.pairwise import cosine_similarity

resnet_sims = []

# Load the Resnet vectors and create a for loop to compare the words pairwise.

# A loop that creates similarities for images pairwise TODO
# for resnet
for w1, w2 in combinations(some_words, 2):
    w1_visfeat = image_features[object_indices[w1]].unsqueeze(0).detach().numpy()
    w2_visfeat = image_features[object_indices[w2]].unsqueeze(0).detach().numpy()
    max_sim = cosine_similarity(w1_visfeat, w2_visfeat)[0][0]
    resnet_sims.append(max_sim)
    print(f"{w1:9} {w2:9} {max_sim:6.3f}")


car       dog        0.779
car       banana     0.749
car       delicious  0.751
car       baguette   0.738
car       jumping    0.752
car       hugging    0.734
car       election   0.752
dog       banana     0.770
dog       delicious  0.786
dog       baguette   0.769
dog       jumping    0.763
dog       hugging    0.803
dog       election   0.802
banana    delicious  0.791
banana    baguette   0.776
banana    jumping    0.749
banana    hugging    0.751
banana    election   0.735
delicious baguette   0.778
delicious jumping    0.787
delicious hugging    0.748
delicious election   0.758
baguette  jumping    0.750
baguette  hugging    0.731
baguette  election   0.746
jumping   hugging    0.765
jumping   election   0.758
hugging   election   0.785


#### Examine if Resnet and GloVe similarities correlate

In [278]:
# a correlation coefficent of two lists
print("Spearman's rho", spearmanr(resnet_sims, glov_sims))

# Higher correlation (closer to 1.0) means two measures agree with each other.

Spearman's rho SpearmanrResult(correlation=0.3459222769567597, pvalue=0.07136834743507675)


How does semantic similarity from word vectors compare with the visual similarity? Are there differences between different words? (+3pt)

In [279]:
# Write your answer here
#As we have seen there might not be so much of a clear similarity between some words that have
#visual similarity but not word-similairty or vice versa. Example of visual similarites, "delicious" for
#spaghetti might also be associated with banana. Here there are very diffrent visual similiarties but very
#similar word similarities, i.e both are "delicious". (see jpg:s)

## Part 3.4 Optional: Examine Fairness In Data Driven Word Vectors

There are no points for this part but you are welcome to further explore this topic if you are inetrested in it. We will address it again in the Computational semantics course.

Caliskan et al. (2017) argues that word vectors learn human biases from data. 

Try to replicate one of the tests of the paper:

Caliskan, Aylin, Joanna J. Bryson, and Arvind Narayanan. “Semantics derived automatically from language corpora contain human-like biases.” Science
356.6334 (2017): 183-186. http://opus.bath.ac.uk/55288/


For example on gender bias:
- Male names: John, Paul, Mike, Kevin, Steve, Greg, Jeff, Bill.
- Female names: Amy, Joan, Lisa, Sarah, Diana, Kate, Ann, Donna.
- Career words : executive, management, professional, corporation, salary, office, business, career.
- Family words : home, parents, children, family, cousins, marriage, wedding, relatives.


Report the average cosine similarity of male names to career words, and compare it with the average similarity of female names to career words. (repeat for family words) 

tokens in GloVe model are all in lower case.

Write at least one sentence to describe your observation.